In [1]:
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import Callback

import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import numpy as np
from time import time
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Restricting GPU memory
os.environ['CUDA_VISIBLE_DEVICES']='0'
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [3]:
import dataloader, TNN, evaluation # Loading helper functions

In [4]:
dev2train, test2train = dataloader.creating_id_mapping()

# Getting i-vectors
trn_bl_ivector, trn_bg_ivector, dev_bl_ivector, dev_bg_ivector, tst_ivector = dataloader.get_ivectors()

# Loading labels for task 2
trn_bl_id, trn_bg_id, dev_bl_id, dev_bg_id, tst_id = dataloader.get_spk_ids()
dev_trials_label = np.append(dev_bl_id,dev_bg_id)

# Making labels for task 1
trn_ivector = np.append(trn_bl_ivector, trn_bg_ivector,axis=0) # combining bg and bl speakers into a single vector
trn_trials = np.append( np.ones([len(trn_bl_ivector), 1]), np.zeros([len(trn_bg_ivector), 1]))
dev_ivector = np.append(dev_bl_ivector, dev_bg_ivector,axis=0) # combining bg and bl
dev_trials = np.append( np.ones([len(dev_bl_id), 1]), np.zeros([len(dev_bg_id), 1]))
tst_trials, tst_trials_label = dataloader.get_tst_trials()

# Models

## TNN model

In [5]:
model, anchor_input, encoded_anchor = TNN.create_TNN()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 600)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 600)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 600)          360600      anchor_input[0][0]               
                                                                 positive_input[0][0]       

## Cosine Similarity

In [6]:
def cosine_similarity(embedding, bl_ivector_embedding, bl_id):
    spk_mean, spk_mean_label = dataloader.make_spkvec(bl_ivector_embedding,bl_id)
    # This accuracy calcultion is provided by MCE2018
    scores = spk_mean.dot(embedding.transpose())
    blscores = spk_mean.dot(bl_ivector_embedding.transpose()) # This will be used in normalization and task 2
    
#     # Multi-target normalization
#     mnorm_mu = np.mean(blscores,axis=1) 
#     mnorm_std = np.std(blscores,axis=1)
#     for iter in range(np.shape(scores)[1]):
#         scores[:,iter]= (scores[:,iter] - mnorm_mu) / mnorm_std
    pred_scores = np.max(scores,axis=0)
    
    # Evaluating Task 2
    identified_label = spk_mean_label[np.argmax(scores,axis=0)]
#     tst_trials_confusion = dataloader.get_trials_label_with_confusion(identified_label, bl_id, id_dict, tst_trials )

    return pred_scores,identified_label

# Training Phase

In [7]:
adam_optim = Adam(lr=1e-5)
model.compile(loss=TNN.triplet_loss, optimizer=adam_optim)

In [8]:
# Preparing the data for triplet mining
all_data = np.column_stack((trn_bl_ivector.reshape(3631,3,600),dev_bl_ivector.reshape(3631,1,600)))
# shape = (classes, number of samples per class, feature size)

# Preparing label for the triplet mining
label_ls = []
for i in range(3631):
    for label in range(4):
        label_ls.append(i)
label_array = np.array(label_ls)

In [9]:
# Defining early stopping so that we mine new triplets when the loss is below 1e-6
# The code is from https://stackoverflow.com/questions/37293642
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
            
callbacks = [EarlyStoppingByLossVal(monitor='loss', value=1e-8, verbose=1)]

In [10]:
total_loss_history = []
total_valloss_history = []

# getting dev ids to train id conversion
dev_bl_id_along_trnset = []
for iter in range(len(dev_bl_id)):
    dev_bl_id_along_trnset.extend([dev2train[dev_bl_id[iter]]])
# model.load_weights('TNN_weights_replicate')
for i in range(100):    
    print('Mining for {}th times'.format(i+1))
    X_train, X_test = TNN.generate_triplet(all_data.reshape(-1,600),label_array,testsize=0.3,ap_pairs=6,an_pairs=50)    
    Anchor = X_train[:,0,:]
    Positive = X_train[:,1,:]
    Negative = X_train[:,2,:]
    Anchor_test = X_test[:,0,:]
    Positive_test = X_test[:,1,:]
    Negative_test = X_test[:,2,:]
    Y_dummy = np.empty((Anchor.shape[0],1))
    Y_dummy2 = np.empty((Anchor_test.shape[0],1))
    history = model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2),
                        batch_size=512, epochs=10, callbacks=callbacks)
    # record down the training history
    total_loss_history = total_loss_history + history.history['loss']
    total_valloss_history = total_valloss_history + history.history['val_loss']
    
    # Making prediction to check the model accuracy
    trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

    transformed_trn_bl_ivector = trained_model.predict(trn_bl_ivector)
    transformed_dev_bl_ivector = trained_model.predict(dev_bl_ivector)
    combined_embedding = np.append(transformed_trn_bl_ivector,transformed_dev_bl_ivector,0)
    combined_labels = np.append(trn_bl_id,dev_bl_id_along_trnset)
    transformed_tst_ivector = trained_model.predict(tst_ivector)

    
    # Task 1
    task1_pred, task2_pred = cosine_similarity(transformed_tst_ivector, combined_embedding, combined_labels)
    err = dataloader.calculate_EER(tst_trials,task1_pred)

    # Task 2
    tst_trials_confusion = dataloader.get_trials_label_with_confusion(task2_pred, tst_trials_label, test2train, tst_trials )
    dev_EER_confusion = dataloader.calculate_EER_with_confusion(task1_pred,tst_trials_confusion)

Mining for 1th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 10s 13us/step - loss: 0.2101 - val_loss: 0.1108
Epoch 2/10
726200/726200 [==============================] - 14s 20us/step - loss: 0.0760 - val_loss: 0.0681
Epoch 3/10
726200/726200 [==============================] - 17s 24us/step - loss: 0.0475 - val_loss: 0.0514
Epoch 4/10
726200/726200 [==============================] - 17s 24us/step - loss: 0.0328 - val_loss: 0.0413
Epoch 5/10
726200/726200 [==============================] - 17s 24us/step - loss: 0.0233 - val_loss: 0.0343
Epoch 6/10
726200/726200 [==============================] - 18s 25us/step - loss: 0.0167 - val_loss: 0.0293
Epoch 7/10
726200/726200 [==============================] - 17s 24us/step - loss: 0.0122 - val_loss: 0.0256
Epoch 8/10
726200/726200 [==============================] - 18s 25us/step - loss: 0.0089 - val_loss: 0.0227
Epoch 9/10
726200/726200 [==============================] - 18s

Epoch 5/10
726200/726200 [==============================] - 17s 24us/step - loss: 8.5430e-05 - val_loss: 3.1386e-04
Epoch 6/10
726200/726200 [==============================] - 17s 24us/step - loss: 5.5419e-05 - val_loss: 3.0353e-04
Epoch 7/10
726200/726200 [==============================] - 17s 24us/step - loss: 3.4255e-05 - val_loss: 2.9574e-04
Epoch 8/10
726200/726200 [==============================] - 17s 24us/step - loss: 1.8885e-05 - val_loss: 2.8912e-04
Epoch 9/10
726200/726200 [==============================] - 17s 24us/step - loss: 9.2037e-06 - val_loss: 2.8573e-04
Epoch 10/10
726200/726200 [==============================] - 18s 24us/step - loss: 4.7043e-06 - val_loss: 2.8211e-04
Top S detector EER is 8.98%
Top 1 detector EER is 13.17% (Total confusion error is 356)
Mining for 8th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 17s 23us/step - loss: 3.0391e-04 - val_loss: 3.1284e-04
Epoch 2/10
726200/726200 [

Epoch 7/10
726200/726200 [==============================] - 17s 24us/step - loss: 1.2196e-06 - val_loss: 1.1035e-04
Epoch 8/10
726200/726200 [==============================] - 17s 23us/step - loss: 2.3324e-07 - val_loss: 1.0944e-04
Epoch 9/10
726200/726200 [==============================] - 17s 24us/step - loss: 9.5978e-09 - val_loss: 1.0927e-04
Epoch 00008: early stopping THR
Top S detector EER is 8.85%
Top 1 detector EER is 12.86% (Total confusion error is 335)
Mining for 14th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 18s 24us/step - loss: 1.4147e-04 - val_loss: 1.2651e-04
Epoch 2/10
726200/726200 [==============================] - 18s 24us/step - loss: 6.8142e-05 - val_loss: 1.1230e-04
Epoch 3/10
726200/726200 [==============================] - 18s 25us/step - loss: 3.8140e-05 - val_loss: 1.0239e-04
Epoch 4/10
726200/726200 [==============================] - 18s 24us/step - loss: 1.9265e-05 - val_loss: 9.581

726200/726200 [==============================] - 9s 12us/step - loss: 8.0912e-05 - val_loss: 7.9703e-05
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.1267e-05 - val_loss: 7.2055e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.3154e-05 - val_loss: 6.6965e-05
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 4.6036e-06 - val_loss: 6.4409e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.7420e-06 - val_loss: 6.3427e-05
Epoch 6/10
726200/726200 [==============================] - 9s 12us/step - loss: 5.3696e-07 - val_loss: 6.2659e-05
Epoch 7/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.0097e-08 - val_loss: 6.2522e-05
Epoch 8/10
726200/726200 [==============================] - 9s 12us/step - loss: 9.4780e-08 - val_loss: 6.2764e-05
Epoch 9/10
726200/726200 [==============================] - 9s 12us/step - loss: 9.8779e-09

726200/726200 [==============================] - 9s 12us/step - loss: 1.3049e-08 - val_loss: 4.0403e-05
Epoch 7/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0000e+00 - val_loss: 4.0403e-05
Epoch 00006: early stopping THR
Top S detector EER is 8.88%
Top 1 detector EER is 12.70% (Total confusion error is 326)
Mining for 28th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 13us/step - loss: 5.9313e-05 - val_loss: 5.1682e-05
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.9777e-05 - val_loss: 4.6009e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 8.6468e-06 - val_loss: 4.3091e-05
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.2407e-06 - val_loss: 3.9800e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 5.4969e-07 - val_loss: 3.7661e-05
Epoch 6/10
7

...

Top S detector EER is 8.88%
Top 1 detector EER is 12.28% (Total confusion error is 309)
Mining for 46th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.5161e-05 - val_loss: 2.7742e-05
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.0360e-05 - val_loss: 2.4425e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.5043e-06 - val_loss: 2.2398e-05
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 6.5733e-07 - val_loss: 2.1900e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 8.6057e-09 - val_loss: 2.1360e-05
Epoch 00004: early stopping THR
Top S detector EER is 8.92%
Top 1 detector EER is 12.32% (Total confusion error is 310)
Mining for 47th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/s

726200/726200 [==============================] - 9s 12us/step - loss: 0.0000e+00 - val_loss: 2.4012e-05
Epoch 00006: early stopping THR
Top S detector EER is 8.94%
Top 1 detector EER is 12.31% (Total confusion error is 305)
Mining for 56th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/step - loss: 2.7806e-05 - val_loss: 2.2802e-05
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 6.3380e-06 - val_loss: 2.0819e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.3916e-06 - val_loss: 2.0129e-05
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.4384e-07 - val_loss: 2.0017e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0000e+00 - val_loss: 2.0016e-05
Epoch 00004: early stopping THR
Top S detector EER is 8.91%
Top 1 detector EER is 12.26% (Total confusion error is 304)
Mining 

Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.2121e-06 - val_loss: 1.3622e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 8.2675e-07 - val_loss: 1.2822e-05
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.4641e-07 - val_loss: 1.2867e-05
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0000e+00 - val_loss: 1.2871e-05
Epoch 00004: early stopping THR
Top S detector EER is 8.84%
Top 1 detector EER is 12.09% (Total confusion error is 296)
Mining for 67th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/step - loss: 2.7937e-05 - val_loss: 2.0311e-05
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 6.1869e-06 - val_loss: 1.7222e-05
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.4420e-06 - val_loss: 1.6061e-05
E

726200/726200 [==============================] - 9s 12us/step - loss: 5.1155e-07 - val_loss: 8.9835e-06
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 7.8022e-08 - val_loss: 8.9752e-06
Epoch 5/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0000e+00 - val_loss: 8.9752e-06
Epoch 00004: early stopping THR
Top S detector EER is 8.81%
Top 1 detector EER is 11.98% (Total confusion error is 298)
Mining for 78th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.8308e-05 - val_loss: 1.0587e-05
Epoch 2/10
726200/726200 [==============================] - 9s 12us/step - loss: 3.1411e-06 - val_loss: 9.4062e-06
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.8028e-07 - val_loss: 9.4255e-06
Epoch 4/10
726200/726200 [==============================] - 9s 12us/step - loss: 0.0000e+00 - val_loss: 9.4255e-06
Epoch 00003:

726200/726200 [==============================] - 9s 13us/step - loss: 0.0000e+00 - val_loss: 7.9231e-06
Epoch 00003: early stopping THR
Top S detector EER is 8.90%
Top 1 detector EER is 12.01% (Total confusion error is 297)
Mining for 90th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [==============================] - 9s 13us/step - loss: 1.5090e-05 - val_loss: 1.0276e-05
Epoch 2/10
726200/726200 [==============================] - 9s 13us/step - loss: 2.4215e-06 - val_loss: 9.1405e-06
Epoch 3/10
726200/726200 [==============================] - 9s 12us/step - loss: 1.9888e-07 - val_loss: 9.1528e-06
Epoch 4/10
726200/726200 [==============================] - 9s 13us/step - loss: 0.0000e+00 - val_loss: 9.1528e-06
Epoch 00003: early stopping THR
Top S detector EER is 8.86%
Top 1 detector EER is 12.04% (Total confusion error is 298)
Mining for 91th times
Train on 726200 samples, validate on 363100 samples
Epoch 1/10
726200/726200 [======================

In [11]:
model.save_weights('TNN_weights_replicate')